# TEST CON FEATURES DEL PSD

# Generar 'X' e 'y' para todos los experimentos
<p>Como datos para las X se toma la media de los 17 canales de:</p>
<ul>
    <li>X_1: PSD de la banda delta (1~4Hz)</li>
    <li>X_2: PSD de la banda theta (4~8Hz)</li>
    <li>X_3: PSD de la banda alpha (8~14Hz)</li>
    <li>X_4: PSD de la banda beta (14~31Hz)</li>
    <li>X_5: PSD de la banda gamma (31~50 Hz)</li>
</ul>

In [ ]:
import mne
from mne.externals.pymatreader import read_mat

from read import read_features

import matplotlib
import matplotlib.pyplot as plt

import numpy as np

import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [ ]:
labels = {
    'leve': 0,
    'moderada': 1,
    'severa': 2
}

def parse_y(y, lim_leve=0.075, lim_moderada=0.15):
    def classify(val):
        if (val <= lim_leve):
            return labels['leve']
        if (val <= lim_moderada):
            return labels['moderada']
        return labels['severa']
    
    len_y = len(y)
    classified = np.zeros(len_y, dtype=int)
    
    for i in range(len_y):
        classified[i] = classify(y[i])
        
    return classified

In [ ]:
# Las que mejores graficas tienen
# features_files = ['4_20151105_noon.mat', '4_20151107_noon.mat', '5_20141108_noon.mat','12_20150928_noon.mat','14_20151014_night.mat', '18_20150926_noon.mat', '21_20151016_noon.mat']

# features_files = ['4_20151105_noon.mat', '5_20141108_noon.mat', '21_20151016_noon.mat']

features_files = ['1_20151124_noon_2.csv', '2_20151106_noon.csv', '3_20151024_noon.csv','4_20151105_noon.csv', '4_20151107_noon.csv',
            '5_20141108_noon.csv', '5_20151012_night.csv', '6_20151121_noon.csv','7_20151015_night.csv', '8_20151022_noon.csv', 
            '9_20151017_night.csv', '10_20151125_noon.csv', '11_20151024_night.csv', '12_20150928_noon.csv', '13_20150929_noon.csv',
            '14_20151014_night.csv','15_20151126_night.csv', '16_20151128_night.csv', '17_20150925_noon.csv', '18_20150926_noon.csv',
            '19_20151114_noon.csv', '20_20151129_night.csv', '21_20151016_noon.csv']

# features_files = ['21_20151016_noon.csv']

X_all = {} # psd
X_all_eog = {} # psd + eog
y_all = {} # datos perclos raw
y_all_limits = {} # thresholds perclos para cada experimento
y_all_class = {} # datos parseados a su clase

for experiment in features_files:
    
    ''' EOG parpadeos por epoch'''
    mat_data = read_mat(f'./SEED-VIG/Raw_Data/{experiment[:-4]}.mat')
    
    sfreq = mat_data['EOG']['eog_config']['current_sample_rate']
    samples = mat_data['EOG']['eog_h']*1e-6 
    samples = np.vstack((samples, mat_data['EOG']['eog_v']*1e-6))

    ch_names = ['EOG_H', 'EOG_V']
    ch_types = ["eog"]*len(ch_names)

    info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)

    info.set_montage('standard_1020')
    
    raw = mne.io.RawArray(samples, info, verbose=False)
    
    # Busqueda del treshold para el parpadeo
    # En los 5 primeros segundos del experimento, buscamos el percentil 2.5 y calculamos su valor absoluto como threshold para detectar parpadeos
    threshold = abs(np.percentile(raw.get_data()[1][:5*1000], 2.5))
    
    blinks = mne.preprocessing.find_eog_events(raw, filter_length='10s', thresh=threshold, verbose=False)
    blinks = blinks.T[0]
    
    n_samples = 885
    dur_sample = 8 # segs
    len_sample = 8*125
    
    blinks_per_sample = []
    for i in range(n_samples):
        blinks_per_sample.append([])
        
    for val in blinks:
        blinks_per_sample[val//len_sample].append(val)
    
    n_blinks_per_sample = [0]*n_samples
        
    for i in range(885):
        n_blinks_per_sample[i] = len(blinks_per_sample[i])
        
    scaler = MinMaxScaler(feature_range=(0, 1))
    n_blinks_per_sample = np.array(n_blinks_per_sample)
    n_blinks_per_sample = scaler.fit_transform(n_blinks_per_sample.reshape(-1,1))
    
    
    ''' PSD '''
    mat_data = read_mat(f'./SEED-VIG/EEG_Feature_5Bands/{experiment[:-4]}.mat')
    perclos_data = read_mat(f'./SEED-VIG/perclos_labels/{experiment[:-4]}.mat')
    y = np.array(perclos_data['perclos'])

    n_channels = 17
    psd_data = mat_data['psd_movingAve']
    perclos = []
    psd_delta = []
    psd_theta = []
    psd_alpha = []
    psd_beta = []
    psd_gamma = []
    start = 0
    end = 885
    for t in range(start, end):
        vals= np.zeros(5)

        for i in range(n_channels):
            vals[0] += psd_data[i][t][0]
            vals[1] += psd_data[i][t][1]
            vals[2] += psd_data[i][t][2]
            vals[3] += psd_data[i][t][3]
            vals[4] += psd_data[i][t][4]
        vals /= 17

        perclos.append(y[t])
        psd_delta.append(vals[0])
        psd_theta.append(vals[1])
        psd_alpha.append(vals[2])
        psd_beta.append(vals[3])
        psd_gamma.append(vals[4])

    
    scaler = MinMaxScaler(feature_range=(0, 1))
    psd_delta = np.array(psd_delta)
    psd_delta = scaler.fit_transform(psd_delta.reshape(-1,1))

    scaler = MinMaxScaler(feature_range=(0, 1))
    psd_theta = np.array(psd_theta)
    psd_theta = scaler.fit_transform(psd_theta.reshape(-1,1))

    scaler = MinMaxScaler(feature_range=(0, 1))
    psd_alpha = np.array(psd_alpha)
    psd_alpha = scaler.fit_transform(psd_alpha.reshape(-1,1))

    scaler = MinMaxScaler(feature_range=(0, 1))
    psd_beta = np.array(psd_beta)
    psd_beta = scaler.fit_transform(psd_beta.reshape(-1,1))
    
    scaler = MinMaxScaler(feature_range=(0, 1))
    psd_gamma = np.array(psd_gamma)
    psd_gamma = scaler.fit_transform(psd_gamma.reshape(-1,1))

    X = psd_delta
    X = np.hstack((X, psd_theta))
    X = np.hstack((X, psd_alpha))
    X = np.hstack((X, psd_beta))
    ''' DESCOMENTAR PARA USAR PSD DE GAMMA '''
    X = np.hstack((X, psd_gamma))
    
    X_all[f'{experiment[:-4]}'] = X # guardamos los psd
    
    X = np.hstack((X, n_blinks_per_sample))
    
    X_all_eog[f'{experiment[:-4]}'] = X # guardamos psd + eog
    
    ''' LABELS '''
    y_all[f'{experiment[:-4]}'] = y # guardamos perclos raw
    
    range_values = (np.max(y)-np.min(y))/100
    lim_leve = np.min(y)+range_values*12.5
    lim_moderada = np.min(y)+range_values*30
    y_all_limits[f'{experiment[:-4]}'] = [lim_leve, lim_moderada]
    
    y_all_class[f'{experiment[:-4]}'] = parse_y(y, lim_leve, lim_moderada) # guardamos ya clasificada
    
print(f'Generados los diccionarios X_all e y_all')

In [ ]:
print(X_all_eog['1_20151124_noon_2'].shape)

for key in X_all:
    print(key)

# Para busqueda de hyperparametros y plotear resultados/metricas

## Clasificacion

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import plot_confusion_matrix

from scipy.stats import uniform, randint, loguniform, gamma

def find_and_test_best_model_class(X, y, model, space, cv=10, n_iter=100, scoring=None, conf_mat=True):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    
    search = RandomizedSearchCV(model, space, n_iter=n_iter, scoring=scoring, n_jobs=-1, cv=cv, random_state=42)
    result = search.fit(X_train, y_train)
    
    print(f"Best Model:")
    print(f"\tScore: {result.best_score_:.4f}")
    print(f"\tHyperparameters: {result.best_params_}")
    
    print('\nResults with X_test:')
    best_model = result.best_estimator_
    final_predictions = best_model.predict(X_test)
    
    #TODO sklearn.metrics.classification_report ofrece los siguientes parametros y alguna cosilla mas, puede ser interesante ???
    acc = accuracy_score(y_test, final_predictions)
    pre = precision_score(y_test, final_predictions, average='macro', zero_division=1)
    rec = recall_score(y_test, final_predictions, average='macro', zero_division=1)
    f1 = f1_score(y_test, final_predictions, average='macro', zero_division=1)
    print(f"\tAccuracy: {acc*100:.2f}%")
    print(f"\tPrecision: {pre:.4f}")
    print(f"\tRecall: {rec:.4f}")
    print(f"\tf1: {f1:.4f}")
    
    ''' PARA MOSTRAR LA MATRIZ DE CONFUSION HAY QUE PASAR EL PARAMETRO conf_mat=True '''
    if (conf_mat):
        print(f'Matriz de confusión para X completo')
        try:
            plot_confusion_matrix(best_model, X, y, display_labels=['leve', 'moderara', 'severa'])
        except:
            pass
    
    return best_model, acc, pre, rec, f1

def map_2_colors(y, labels):
    label_2_color = {
        0: 'green',
        1: 'orange',
        2: 'red'
    }
    
    cmap = []
    for i in range(len(y)):
        cmap.append(label_2_color[y[i]])
    return cmap

def plot_results_class(X, y, model, lim_leve_moderada, lim_moderada_severa):
    plt.figure(figsize=(20, 8))
    plt.axhspan(0, lim_leve_moderada, facecolor='green', alpha=0.2, label="Somnolencia leve")
    plt.axhspan(lim_leve_moderada, lim_moderada_severa, facecolor='orange', alpha=0.2, label="Somnolencia moderada")
    plt.axhspan(lim_moderada_severa, 1, facecolor='red', alpha=0.2, label="Somnolencia severa")
    plt.plot(y, c='k', alpha=0.6)
    colors = map_2_colors(model.predict(X), labels)
    plt.scatter(range(len(y)), y, c=colors)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

## Juntamos todos los datos en un solo X e y

In [ ]:
from read import read_features

X = []
X_eog = []
X_eeg = []
y = []
y_class = []

for key in X_all:
    X.append(X_all[key])
    X_eog.append(X_all_eog[key])
    aux, aux2 = read_features(f'./features/{key}.csv')
    scaler = MinMaxScaler(feature_range=(0, 1))
    aux_minmax = scaler.fit_transform(aux)
    X_eeg.append(aux_minmax)
    
    y.append(y_all[key])
    y_class.append(y_all_class[key])

X = np.array(X)
X = X.reshape((X.shape[0]*X.shape[1], X.shape[2]))
X_eog = np.array(X_eog)
X_eog = X_eog.reshape((X_eog.shape[0]*X_eog.shape[1], X_eog.shape[2]))
X_eeg = np.array(X_eeg)
X_eeg = X_eeg.reshape((X_eeg.shape[0]*X_eeg.shape[1], X_eeg.shape[2]))

y = np.array(y)
y = y.reshape((y.shape[0]*y.shape[1]))
y_class = np.array(y_class)
y_class = y_class.reshape((y_class.shape[0]*y_class.shape[1]))

In [ ]:
print(f'{X.shape}, {X_eog.shape}, {X_eeg.shape}, {y.shape}, {y_class.shape}')

# Modelo General
Se entrena con todos los datos de los experimentos

## SMV-c

In [ ]:
from sklearn.svm import SVC

### SMV-c EEG

In [ ]:
print(f"Experiment: Modelo General SVM-c EEG")
model = SVC(random_state=42)

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

space = dict()
space['C'] = uniform(1, 20)
space['kernel'] = ['linear', 'rbf']
space['gamma'] = ['scale', 'auto']
space['shrinking'] = [True, False]
space['probability'] = [True, False]
space['decision_function_shape'] = ['ovo', 'ovr']

general_model, acc, pre, rec, f1 = find_and_test_best_model_class(X_eeg, y_class, model, space, cv=cv,  n_iter=10)

In [ ]:
for key in X_all:
    data, aux2 = read_features(f'./features/{key}.csv')
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_minmax = scaler.fit_transform(data)
    
    perclos = y_all[key]
    expected = y_all_class[key]
    
    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    
    print(f'Experiment: {key}')
    
    plt.figure(figsize=(20, 8))
    plt.axhspan(0, lim_leve_moderada, facecolor='green', alpha=0.2, label="Somnolencia leve")
    plt.axhspan(lim_leve_moderada, lim_moderada_severa, facecolor='orange', alpha=0.2, label="Somnolencia moderada")
    plt.axhspan(lim_moderada_severa, 1, facecolor='red', alpha=0.2, label="Somnolencia severa")
    plt.plot(perclos, c='k', alpha=0.6)
    colors = map_2_colors(general_model.predict(data_minmax), labels)
    plt.scatter(range(len(perclos)), perclos, c=colors)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

### SMV-c PSD

In [ ]:
print(f"Experiment: Modelo General SVM-c PSD")
model = SVC(random_state=42)

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

space = dict()
space['C'] = uniform(1, 20)
space['kernel'] = ['linear', 'rbf']
space['gamma'] = ['scale', 'auto']
space['shrinking'] = [True, False]
space['probability'] = [True, False]
space['decision_function_shape'] = ['ovo', 'ovr']

general_model, acc, pre, rec, f1 = find_and_test_best_model_class(X, y_class, model, space, cv=cv,  n_iter=10)

In [ ]:
for key in X_all:
    data = X_all[key]
    
    perclos = y_all[key]
    expected = y_all_class[key]
    
    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    
    print(f'Experiment: {key}')
    
    plt.figure(figsize=(20, 8))
    plt.axhspan(0, lim_leve_moderada, facecolor='green', alpha=0.2, label="Somnolencia leve")
    plt.axhspan(lim_leve_moderada, lim_moderada_severa, facecolor='orange', alpha=0.2, label="Somnolencia moderada")
    plt.axhspan(lim_moderada_severa, 1, facecolor='red', alpha=0.2, label="Somnolencia severa")
    plt.plot(perclos, c='k', alpha=0.6)
    colors = map_2_colors(general_model.predict(data), labels)
    plt.scatter(range(len(perclos)), perclos, c=colors)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

### SMV-c PSD + EOG

In [ ]:
print(f"Experiment: Modelo General SVM-c PSD + EOG")
model = SVC(random_state=42)

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

space = dict()
space['C'] = uniform(1, 20)
space['kernel'] = ['linear', 'rbf']
space['gamma'] = ['scale', 'auto']
space['shrinking'] = [True, False]
space['probability'] = [True, False]
space['decision_function_shape'] = ['ovo', 'ovr']

general_model, acc, pre, rec, f1 = find_and_test_best_model_class(X_eog, y_class, model, space, cv=cv,  n_iter=10)

In [ ]:
for key in X_all:
    data = X_all_eog[key]
    
    perclos = y_all[key]
    expected = y_all_class[key]
    
    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    
    print(f'Experiment: {key}')
    
    plt.figure(figsize=(20, 8))
    plt.axhspan(0, lim_leve_moderada, facecolor='green', alpha=0.2, label="Somnolencia leve")
    plt.axhspan(lim_leve_moderada, lim_moderada_severa, facecolor='orange', alpha=0.2, label="Somnolencia moderada")
    plt.axhspan(lim_moderada_severa, 1, facecolor='red', alpha=0.2, label="Somnolencia severa")
    plt.plot(perclos, c='k', alpha=0.6)
    colors = map_2_colors(general_model.predict(data), labels)
    plt.scatter(range(len(perclos)), perclos, c=colors)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

### KNN EEG

In [ ]:
print(f"Experiment: Modelo General KNN EEG")
model = KNeighborsClassifier()

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

space = dict()
space['n_neighbors'] = randint(3, 21)
space['weights'] = ['uniform', 'distance']
space['algorithm'] = ['auto', 'ball_tree', 'kd_tree', 'brute']
space['leaf_size'] = randint(15,50)
space['p'] = [1, 2]

general_model, acc, pre, rec, f1 = find_and_test_best_model_class(X_eeg, y_class, model, space, cv=cv,  n_iter=5)

In [ ]:
for key in X_all:
    data, aux2 = read_features(f'./features/{key}.csv')
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_minmax = scaler.fit_transform(data)
    
    perclos = y_all[key]
    expected = y_all_class[key]
    
    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    
    print(f'Experiment: {key}')
    
    plt.figure(figsize=(20, 8))
    plt.axhspan(0, lim_leve_moderada, facecolor='green', alpha=0.2, label="Somnolencia leve")
    plt.axhspan(lim_leve_moderada, lim_moderada_severa, facecolor='orange', alpha=0.2, label="Somnolencia moderada")
    plt.axhspan(lim_moderada_severa, 1, facecolor='red', alpha=0.2, label="Somnolencia severa")
    plt.plot(perclos, c='k', alpha=0.6)
    colors = map_2_colors(general_model.predict(data_minmax), labels)
    plt.scatter(range(len(perclos)), perclos, c=colors)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

### KNN PSD

In [ ]:
print(f"Experiment: Modelo General KNN PSD")
model = KNeighborsClassifier()

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

space = dict()
space['n_neighbors'] = randint(3, 21)
space['weights'] = ['uniform', 'distance']
space['algorithm'] = ['auto', 'ball_tree', 'kd_tree', 'brute']
space['leaf_size'] = randint(15,50)
space['p'] = [1, 2]

general_model, acc, pre, rec, f1 = find_and_test_best_model_class(X, y_class, model, space, cv=cv,  n_iter=10)

In [ ]:
for key in X_all:
    data = X_all[key]
    
    perclos = y_all[key]
    expected = y_all_class[key]
    
    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    
    print(f'Experiment: {key}')
    
    plt.figure(figsize=(20, 8))
    plt.axhspan(0, lim_leve_moderada, facecolor='green', alpha=0.2, label="Somnolencia leve")
    plt.axhspan(lim_leve_moderada, lim_moderada_severa, facecolor='orange', alpha=0.2, label="Somnolencia moderada")
    plt.axhspan(lim_moderada_severa, 1, facecolor='red', alpha=0.2, label="Somnolencia severa")
    plt.plot(perclos, c='k', alpha=0.6)
    colors = map_2_colors(general_model.predict(data), labels)
    plt.scatter(range(len(perclos)), perclos, c=colors)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

### KNN PSD + EOG

In [ ]:
print(f"Experiment: Modelo General KNN PSD + EOG")
model = KNeighborsClassifier()

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

space = dict()
space['n_neighbors'] = randint(3, 21)
space['weights'] = ['uniform', 'distance']
space['algorithm'] = ['auto', 'ball_tree', 'kd_tree', 'brute']
space['leaf_size'] = randint(15,50)
space['p'] = [1, 2]

general_model, acc, pre, rec, f1 = find_and_test_best_model_class(X_eog, y_class, model, space, cv=cv,  n_iter=10)

In [ ]:
for key in X_all:
    data = X_all_eog[key]
    
    perclos = y_all[key]
    expected = y_all_class[key]
    
    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    
    print(f'Experiment: {key}')
    
    plt.figure(figsize=(20, 8))
    plt.axhspan(0, lim_leve_moderada, facecolor='green', alpha=0.2, label="Somnolencia leve")
    plt.axhspan(lim_leve_moderada, lim_moderada_severa, facecolor='orange', alpha=0.2, label="Somnolencia moderada")
    plt.axhspan(lim_moderada_severa, 1, facecolor='red', alpha=0.2, label="Somnolencia severa")
    plt.plot(perclos, c='k', alpha=0.6)
    colors = map_2_colors(general_model.predict(data), labels)
    plt.scatter(range(len(perclos)), perclos, c=colors)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

### Decision tree EEG

In [ ]:
print(f"Experiment: Modelo General Decision Tree PSD")
model = DecisionTreeClassifier(random_state=42)

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

space = dict()
space['criterion'] = ["gini", "entropy"]
space['splitter'] = ["best", "random"]
space['max_depth'] = randint(5, 100)
space['min_samples_split'] = randint(2, 10)
#space['min_samples_leaf'] = randint(1, 10)
space['max_features'] = ["auto", "sqrt", "log2"]
#space['ccp_alpha'] = uniform(0, 5) # empeora el ponerlo, el mejor valor es 0.0

general_model, acc, pre, rec, f1 = find_and_test_best_model_class(X_eeg, y_class, model, space, cv=cv,  n_iter=20)

In [ ]:
for key in X_all:
    data, aux2 = read_features(f'./features/{key}.csv')
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_minmax = scaler.fit_transform(data)
    
    perclos = y_all[key]
    expected = y_all_class[key]
    
    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    
    print(f'Experiment: {key}')
    
    plt.figure(figsize=(20, 8))
    plt.axhspan(0, lim_leve_moderada, facecolor='green', alpha=0.2, label="Somnolencia leve")
    plt.axhspan(lim_leve_moderada, lim_moderada_severa, facecolor='orange', alpha=0.2, label="Somnolencia moderada")
    plt.axhspan(lim_moderada_severa, 1, facecolor='red', alpha=0.2, label="Somnolencia severa")
    plt.plot(perclos, c='k', alpha=0.6)
    colors = map_2_colors(general_model.predict(data_minmax), labels)
    plt.scatter(range(len(perclos)), perclos, c=colors)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

### Decision tree PSD

In [ ]:
print(f"Experiment: Modelo General Decision Tree PSD")
model = DecisionTreeClassifier(random_state=42)

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

space = dict()
space['criterion'] = ["gini", "entropy"]
space['splitter'] = ["best", "random"]
space['max_depth'] = randint(5, 100)
space['min_samples_split'] = randint(2, 10)
#space['min_samples_leaf'] = randint(1, 10)
space['max_features'] = ["auto", "sqrt", "log2"]
#space['ccp_alpha'] = uniform(0, 5) # empeora el ponerlo, el mejor valor es 0.0

general_model, acc, pre, rec, f1 = find_and_test_best_model_class(X, y_class, model, space, cv=cv,  n_iter=10)

In [ ]:
attribs = ['psd_delta', 'psd_theta', 'psd_alpha', 'psd_betha', 'psd_gamma', 'n_parpadeos']
importances = general_model.feature_importances_
importance_features = sorted(zip(importances, attribs), reverse=True)
for imp, attr in importance_features:
    print(f'{attr}: {imp:.4f}')

In [ ]:
for key in X_all:
    data = X_all[key]
    
    perclos = y_all[key]
    expected = y_all_class[key]
    
    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    
    print(f'Experiment: {key}')
    
    plt.figure(figsize=(20, 8))
    plt.axhspan(0, lim_leve_moderada, facecolor='green', alpha=0.2, label="Somnolencia leve")
    plt.axhspan(lim_leve_moderada, lim_moderada_severa, facecolor='orange', alpha=0.2, label="Somnolencia moderada")
    plt.axhspan(lim_moderada_severa, 1, facecolor='red', alpha=0.2, label="Somnolencia severa")
    plt.plot(perclos, c='k', alpha=0.6)
    colors = map_2_colors(general_model.predict(data), labels)
    plt.scatter(range(len(perclos)), perclos, c=colors)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

### Decision Tree PSD + EOG

In [ ]:
print(f"Experiment: Modelo General Decision Tree PSD + EOG")
model = DecisionTreeClassifier(random_state=42)

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

space = dict()
space['criterion'] = ["gini", "entropy"]
space['splitter'] = ["best", "random"]
space['max_depth'] = randint(5, 100)
space['min_samples_split'] = randint(2, 10)
#space['min_samples_leaf'] = randint(1, 10)
space['max_features'] = ["auto", "sqrt", "log2"]
#space['ccp_alpha'] = uniform(0, 5) # empeora el ponerlo, el mejor valor es 0.0

general_model, acc, pre, rec, f1 = find_and_test_best_model_class(X_eog, y_class, model, space, cv=cv,  n_iter=10)

In [ ]:
attribs = ['psd_delta', 'psd_theta', 'psd_alpha', 'psd_betha', 'psd_gamma', 'n_parpadeos']
importances = general_model.feature_importances_
importance_features = sorted(zip(importances, attribs), reverse=True)
for imp, attr in importance_features:
    print(f'{attr}: {imp:.4f}')

In [ ]:
for key in X_all:
    data = X_all_eog[key]
    
    perclos = y_all[key]
    expected = y_all_class[key]
    
    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    
    print(f'Experiment: {key}')
    
    plt.figure(figsize=(20, 8))
    plt.axhspan(0, lim_leve_moderada, facecolor='green', alpha=0.2, label="Somnolencia leve")
    plt.axhspan(lim_leve_moderada, lim_moderada_severa, facecolor='orange', alpha=0.2, label="Somnolencia moderada")
    plt.axhspan(lim_moderada_severa, 1, facecolor='red', alpha=0.2, label="Somnolencia severa")
    plt.plot(perclos, c='k', alpha=0.6)
    colors = map_2_colors(general_model.predict(data), labels)
    plt.scatter(range(len(perclos)), perclos, c=colors)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

### Random Forest EEG

In [ ]:
print(f"Experiment: Modelo General Random Forest EEG")
model = RandomForestClassifier(random_state=42, n_jobs=-1)

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

space = dict()
space['n_estimators'] = randint(75, 200)
space['criterion'] = ["gini", "entropy"]
space['max_depth'] = randint(75, 150)
space['min_samples_split'] = randint(2, 10)
space['min_samples_leaf'] = randint(1, 10)
# space['min_weight_fraction_leaf'] = uniform(0, 0.5)
space['max_features'] = ["auto", "sqrt", "log2"]
#space['max_leaf_nodes'] = randint(1, 300)
# space['min_impurity_decrease'] = uniform(0, 5)
#space['bootstrap'] = [True, False]
# space['oob_score'] = [True, False]
space['warm_start'] = [True, False]
# space['class_weight'] = ['balanced', 'balanced_subsample']
#space['ccp_alpha'] = uniform(0, 5)
#space['max_samples'] = uniform(0, 1)

general_model, acc, pre, rec, f1 = find_and_test_best_model_class(X_eeg, y_class, model, space, cv=cv,  n_iter=20)

In [ ]:
for key in X_all:
    data, aux2 = read_features(f'./features/{key}.csv')
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_minmax = scaler.fit_transform(data)
    
    perclos = y_all[key]
    expected = y_all_class[key]
    
    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    
    print(f'Experiment: {key}')
    
    plt.figure(figsize=(20, 8))
    plt.axhspan(0, lim_leve_moderada, facecolor='green', alpha=0.2, label="Somnolencia leve")
    plt.axhspan(lim_leve_moderada, lim_moderada_severa, facecolor='orange', alpha=0.2, label="Somnolencia moderada")
    plt.axhspan(lim_moderada_severa, 1, facecolor='red', alpha=0.2, label="Somnolencia severa")
    plt.plot(perclos, c='k', alpha=0.6)
    colors = map_2_colors(general_model.predict(data_minmax), labels)
    plt.scatter(range(len(perclos)), perclos, c=colors)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

### Random Forest PSD

In [ ]:
print(f"Experiment: Modelo General Random Forest PSD")
model = RandomForestClassifier(random_state=42, n_jobs=-1)

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

space = dict()
space['n_estimators'] = randint(75, 200)
space['criterion'] = ["gini", "entropy"]
space['max_depth'] = randint(75, 150)
space['min_samples_split'] = randint(2, 10)
space['min_samples_leaf'] = randint(1, 10)
# space['min_weight_fraction_leaf'] = uniform(0, 0.5)
space['max_features'] = ["auto", "sqrt", "log2"]
#space['max_leaf_nodes'] = randint(1, 300)
# space['min_impurity_decrease'] = uniform(0, 5)
#space['bootstrap'] = [True, False]
# space['oob_score'] = [True, False]
space['warm_start'] = [True, False]
# space['class_weight'] = ['balanced', 'balanced_subsample']
#space['ccp_alpha'] = uniform(0, 5)
#space['max_samples'] = uniform(0, 1)

general_model, acc, pre, rec, f1 = find_and_test_best_model_class(X, y_class, model, space, cv=cv,  n_iter=20)

In [ ]:
attribs = ['psd_delta', 'psd_theta', 'psd_alpha', 'psd_betha', 'psd_gamma', 'n_parpadeos']
importances = general_model.feature_importances_
importance_features = sorted(zip(importances, attribs), reverse=True)
for imp, attr in importance_features:
    print(f'{attr}: {imp:.4f}')

In [ ]:
for key in X_all:
    data = X_all[key]
    
    perclos = y_all[key]
    expected = y_all_class[key]
    
    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    
    print(f'Experiment: {key}')
    
    plt.figure(figsize=(20, 8))
    plt.axhspan(0, lim_leve_moderada, facecolor='green', alpha=0.2, label="Somnolencia leve")
    plt.axhspan(lim_leve_moderada, lim_moderada_severa, facecolor='orange', alpha=0.2, label="Somnolencia moderada")
    plt.axhspan(lim_moderada_severa, 1, facecolor='red', alpha=0.2, label="Somnolencia severa")
    plt.plot(perclos, c='k', alpha=0.6)
    colors = map_2_colors(general_model.predict(data), labels)
    plt.scatter(range(len(perclos)), perclos, c=colors)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

### Random Forest PSD + EOG

In [ ]:
print(f"Experiment: Modelo General Random Forest PSD + EOG")
model = RandomForestClassifier(random_state=42, n_jobs=-1)

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

space = dict()
space['n_estimators'] = randint(75, 200)
space['criterion'] = ["gini", "entropy"]
space['max_depth'] = randint(75, 150)
space['min_samples_split'] = randint(2, 10)
space['min_samples_leaf'] = randint(1, 10)
# space['min_weight_fraction_leaf'] = uniform(0, 0.5)
space['max_features'] = ["auto", "sqrt", "log2"]
#space['max_leaf_nodes'] = randint(1, 300)
# space['min_impurity_decrease'] = uniform(0, 5)
#space['bootstrap'] = [True, False]
# space['oob_score'] = [True, False]
space['warm_start'] = [True, False]
# space['class_weight'] = ['balanced', 'balanced_subsample']
#space['ccp_alpha'] = uniform(0, 5)
#space['max_samples'] = uniform(0, 1)

general_model, acc, pre, rec, f1 = find_and_test_best_model_class(X_eog, y_class, model, space, cv=cv,  n_iter=20)

In [ ]:
attribs = ['psd_delta', 'psd_theta', 'psd_alpha', 'psd_betha', 'psd_gamma', 'n_parpadeos']
importances = general_model.feature_importances_
importance_features = sorted(zip(importances, attribs), reverse=True)
for imp, attr in importance_features:
    print(f'{attr}: {imp:.4f}')

In [ ]:
for key in X_all:
    data = X_all_eog[key]
    
    perclos = y_all[key]
    expected = y_all_class[key]
    
    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    
    print(f'Experiment: {key}')
    
    plt.figure(figsize=(20, 8))
    plt.axhspan(0, lim_leve_moderada, facecolor='green', alpha=0.2, label="Somnolencia leve")
    plt.axhspan(lim_leve_moderada, lim_moderada_severa, facecolor='orange', alpha=0.2, label="Somnolencia moderada")
    plt.axhspan(lim_moderada_severa, 1, facecolor='red', alpha=0.2, label="Somnolencia severa")
    plt.plot(perclos, c='k', alpha=0.6)
    colors = map_2_colors(general_model.predict(data), labels)
    plt.scatter(range(len(perclos)), perclos, c=colors)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

# Modelo 90-10
- 21/23 = ~9%
- 2/23 = ~10%X_eog

In [ ]:
# Separamos 2 experimentos para hacer las pruebas
subset_train, subset_test = train_test_split(features_files, test_size=2, random_state=77)

X_sub = []
X_sub_eog = []
X_sub_eeg = []
y_sub = []
y_sub_class = []

for key in subset_train:
    X_sub.append(X_all[key[:-4]])
    X_sub_eog.append(X_all_eog[key[:-4]])
    aux, aux2 = read_features(f'./features/{key[:-4]}.csv')
    aux_minmax = scaler.fit_transform(aux)
    X_sub_eeg.append(aux_minmax)
    
    y_sub.append(y_all[key[:-4]])
    y_sub_class.append(y_all_class[key[:-4]])

X_sub = np.array(X_sub)
X_sub = X_sub.reshape((X_sub.shape[0]*X_sub.shape[1], X_sub.shape[2]))
X_sub_eog = np.array(X_sub_eog)
X_sub_eog = X_sub_eog.reshape((X_sub_eog.shape[0]*X_sub_eog.shape[1], X_sub_eog.shape[2]))
X_sub_eeg = np.array(X_sub_eeg)
X_sub_eeg = X_sub_eeg.reshape((X_sub_eeg.shape[0]*X_sub_eeg.shape[1], X_sub_eeg.shape[2]))

y_sub = np.array(y_sub)
y_sub = y_sub.reshape((y_sub.shape[0]*y_sub.shape[1]))
y_sub_class = np.array(y_sub_class)
y_sub_class = y_sub_class.reshape((y_sub_class.shape[0]*y_sub_class.shape[1]))

In [ ]:
print(f'{X_sub.shape}, {X_sub_eog.shape}, {X_sub_eeg.shape}, {y_sub.shape}, {y_sub_class.shape}')

## SMV-c

In [ ]:
from sklearn.svm import SVC

### SMV-c PSD

In [ ]:
print(f"Experiment: Modelo 90-10 SVM-c PSD")
model = SVC(random_state=42)

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

space = dict()
space['C'] = uniform(1, 20)
space['kernel'] = ['linear', 'rbf']
space['gamma'] = ['scale', 'auto']
space['shrinking'] = [True, False]
space['probability'] = [True, False]
space['decision_function_shape'] = ['ovo', 'ovr']

partial_model, acc, pre, rec, f1 = find_and_test_best_model_class(X_sub, y_sub_class, model, space, cv=cv,  n_iter=10)

In [ ]:
for experiment in subset_test:
    key = f'{experiment[:-4]}'
    data = X_all[key]
    
    perclos = y_all[key]
    expected = y_all_class[key]
    
    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    
    print(f'Experiment: {key}')
    
    plt.figure(figsize=(20, 8))
    plt.axhspan(0, lim_leve_moderada, facecolor='green', alpha=0.2, label="Somnolencia leve")
    plt.axhspan(lim_leve_moderada, lim_moderada_severa, facecolor='orange', alpha=0.2, label="Somnolencia moderada")
    plt.axhspan(lim_moderada_severa, 1, facecolor='red', alpha=0.2, label="Somnolencia severa")
    plt.plot(perclos, c='k', alpha=0.6)
    colors = map_2_colors(partial_model.predict(data), labels)
    plt.scatter(range(len(perclos)), perclos, c=colors)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

### SMV-r PSD + EOG

In [ ]:
print(f"Experiment: Modelo 90-10 SVM-r PSD + EOG")
model = SVC(random_state=42)

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

space = dict()
space['C'] = uniform(1, 20)
space['kernel'] = ['linear', 'rbf']
space['gamma'] = ['scale', 'auto']
space['shrinking'] = [True, False]
space['probability'] = [True, False]
space['decision_function_shape'] = ['ovo', 'ovr']

partial_model, acc, pre, rec, f1 = find_and_test_best_model_class(X_sub_eog, y_sub_class, model, space, cv=cv,  n_iter=10)

In [ ]:
for experiment in subset_test:
    key = f'{experiment[:-4]}'
    data = X_all_eog[key]
    
    perclos = y_all[key]
    expected = y_all_class[key]
    
    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    
    print(f'Experiment: {key}')
    
    plt.figure(figsize=(20, 8))
    plt.axhspan(0, lim_leve_moderada, facecolor='green', alpha=0.2, label="Somnolencia leve")
    plt.axhspan(lim_leve_moderada, lim_moderada_severa, facecolor='orange', alpha=0.2, label="Somnolencia moderada")
    plt.axhspan(lim_moderada_severa, 1, facecolor='red', alpha=0.2, label="Somnolencia severa")
    plt.plot(perclos, c='k', alpha=0.6)
    colors = map_2_colors(partial_model.predict(data), labels)
    plt.scatter(range(len(perclos)), perclos, c=colors)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

### KNN PSD

In [ ]:
print(f"Experiment: Modelo 90-10 KNN PSD")
model = KNeighborsClassifier()

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

space = dict()
space['n_neighbors'] = randint(3, 21)
space['weights'] = ['uniform', 'distance']
space['algorithm'] = ['auto', 'ball_tree', 'kd_tree', 'brute']
space['leaf_size'] = randint(15,50)
space['p'] = [1, 2]

partial_model, acc, pre, rec, f1 = find_and_test_best_model_class(X_sub, y_sub_class, model, space, cv=cv,  n_iter=20)

In [ ]:
for experiment in subset_test:
    key = f'{experiment[:-4]}'
    data = X_all[key]
    
    perclos = y_all[key]
    expected = y_all_class[key]
    
    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    
    print(f'Experiment: {key}')
    
    plt.figure(figsize=(20, 8))
    plt.axhspan(0, lim_leve_moderada, facecolor='green', alpha=0.2, label="Somnolencia leve")
    plt.axhspan(lim_leve_moderada, lim_moderada_severa, facecolor='orange', alpha=0.2, label="Somnolencia moderada")
    plt.axhspan(lim_moderada_severa, 1, facecolor='red', alpha=0.2, label="Somnolencia severa")
    plt.plot(perclos, c='k', alpha=0.6)
    colors = map_2_colors(partial_model.predict(data), labels)
    plt.scatter(range(len(perclos)), perclos, c=colors)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

### KNN PSD + EOG

In [ ]:
print(f"Experiment: Modelo 90-10 KNN PSD+EOG")
model = KNeighborsClassifier()

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

space = dict()
space['n_neighbors'] = randint(3, 21)
space['weights'] = ['uniform', 'distance']
space['algorithm'] = ['auto', 'ball_tree', 'kd_tree', 'brute']
space['leaf_size'] = randint(15,50)
space['p'] = [1, 2]

partial_model, acc, pre, rec, f1 = find_and_test_best_model_class(X_sub_eog, y_sub_class, model, space, cv=cv,  n_iter=20)

In [ ]:
for experiment in subset_test:
    key = f'{experiment[:-4]}'
    data = X_all_eog[key]
    
    perclos = y_all[key]
    expected = y_all_class[key]
    
    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    
    print(f'Experiment: {key}')
    
    plt.figure(figsize=(20, 8))
    plt.axhspan(0, lim_leve_moderada, facecolor='green', alpha=0.2, label="Somnolencia leve")
    plt.axhspan(lim_leve_moderada, lim_moderada_severa, facecolor='orange', alpha=0.2, label="Somnolencia moderada")
    plt.axhspan(lim_moderada_severa, 1, facecolor='red', alpha=0.2, label="Somnolencia severa")
    plt.plot(perclos, c='k', alpha=0.6)
    colors = map_2_colors(partial_model.predict(data), labels)
    plt.scatter(range(len(perclos)), perclos, c=colors)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

### Decision Tree PSD

In [ ]:
print(f"Experiment: Modelo 90-10 Decision Tree PSD")
model = DecisionTreeClassifier(random_state=42)

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

space = dict()
space['criterion'] = ["gini", "entropy"]
space['splitter'] = ["best", "random"]
space['max_depth'] = randint(5, 100)
space['min_samples_split'] = randint(2, 10)
#space['min_samples_leaf'] = randint(1, 10)
space['max_features'] = ["auto", "sqrt", "log2"]
#space['ccp_alpha'] = uniform(0, 5) # empeora el ponerlo, el mejor valor es 0.0

partial_model, acc, pre, rec, f1 = find_and_test_best_model_class(X_sub, y_sub_class, model, space, cv=cv,  n_iter=100)

In [ ]:
attribs = ['psd_delta', 'psd_theta', 'psd_alpha', 'psd_betha', 'psd_gamma', 'n_parpadeos']
importances = partial_model.feature_importances_
importance_features = sorted(zip(importances, attribs), reverse=True)
for imp, attr in importance_features:
    print(f'{attr}: {imp:.4f}')

In [ ]:
for experiment in subset_test:
    key = f'{experiment[:-4]}'
    data = X_all[key]
    
    perclos = y_all[key]
    expected = y_all_class[key]
    
    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    
    print(f'Experiment: {key}')
    
    plt.figure(figsize=(20, 8))
    plt.axhspan(0, lim_leve_moderada, facecolor='green', alpha=0.2, label="Somnolencia leve")
    plt.axhspan(lim_leve_moderada, lim_moderada_severa, facecolor='orange', alpha=0.2, label="Somnolencia moderada")
    plt.axhspan(lim_moderada_severa, 1, facecolor='red', alpha=0.2, label="Somnolencia severa")
    plt.plot(perclos, c='k', alpha=0.6)
    colors = map_2_colors(partial_model.predict(data), labels)
    plt.scatter(range(len(perclos)), perclos, c=colors)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

### Decision Tree PSD + EOG

In [ ]:
print(f"Experiment: Modelo 90-10 Decision Tree PSD")
model = DecisionTreeClassifier(random_state=42)

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

space = dict()
space['criterion'] = ["gini", "entropy"]
space['splitter'] = ["best", "random"]
space['max_depth'] = randint(5, 100)
space['min_samples_split'] = randint(2, 10)
#space['min_samples_leaf'] = randint(1, 10)
space['max_features'] = ["auto", "sqrt", "log2"]
#space['ccp_alpha'] = uniform(0, 5) # empeora el ponerlo, el mejor valor es 0.0

partial_model, acc, pre, rec, f1 = find_and_test_best_model_class(X_sub_eog, y_sub_class, model, space, cv=cv,  n_iter=100)

In [ ]:
attribs = ['psd_delta', 'psd_theta', 'psd_alpha', 'psd_betha', 'psd_gamma', 'n_parpadeos']
importances = partial_model.feature_importances_
importance_features = sorted(zip(importances, attribs), reverse=True)
for imp, attr in importance_features:
    print(f'{attr}: {imp:.4f}')

In [ ]:
for experiment in subset_test:
    key = f'{experiment[:-4]}'
    data = X_all_eog[key]
    
    perclos = y_all[key]
    expected = y_all_class[key]
    
    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    
    print(f'Experiment: {key}')
    
    plt.figure(figsize=(20, 8))
    plt.axhspan(0, lim_leve_moderada, facecolor='green', alpha=0.2, label="Somnolencia leve")
    plt.axhspan(lim_leve_moderada, lim_moderada_severa, facecolor='orange', alpha=0.2, label="Somnolencia moderada")
    plt.axhspan(lim_moderada_severa, 1, facecolor='red', alpha=0.2, label="Somnolencia severa")
    plt.plot(perclos, c='k', alpha=0.6)
    colors = map_2_colors(partial_model.predict(data), labels)
    plt.scatter(range(len(perclos)), perclos, c=colors)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

### Random Forest PSD

In [ ]:
print(f"Experiment: Modelo 90-10 Random Forest PSD")
model = RandomForestClassifier(random_state=42, n_jobs=-1)

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

space = dict()
space['n_estimators'] = randint(75, 200)
space['criterion'] = ["gini", "entropy"]
space['max_depth'] = randint(75, 150)
space['min_samples_split'] = randint(2, 10)
space['min_samples_leaf'] = randint(1, 10)
# space['min_weight_fraction_leaf'] = uniform(0, 0.5)
space['max_features'] = ["auto", "sqrt", "log2"]
#space['max_leaf_nodes'] = randint(1, 300)
# space['min_impurity_decrease'] = uniform(0, 5)
#space['bootstrap'] = [True, False]
# space['oob_score'] = [True, False]
space['warm_start'] = [True, False]
# space['class_weight'] = ['balanced', 'balanced_subsample']
#space['ccp_alpha'] = uniform(0, 5)
#space['max_samples'] = uniform(0, 1)

partial_model, acc, pre, rec, f1 = find_and_test_best_model_class(X_sub, y_sub_class, model, space, cv=cv,  n_iter=20)

In [ ]:
attribs = ['psd_delta', 'psd_theta', 'psd_alpha', 'psd_betha', 'psd_gamma', 'n_parpadeos']
importances = partial_model.feature_importances_
importance_features = sorted(zip(importances, attribs), reverse=True)
for imp, attr in importance_features:
    print(f'{attr}: {imp:.4f}')

In [ ]:
for experiment in subset_test:
    key = f'{experiment[:-4]}'
    data = X_all[key]
    
    perclos = y_all[key]
    expected = y_all_class[key]
    
    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    
    print(f'Experiment: {key}')
    
    plt.figure(figsize=(20, 8))
    plt.axhspan(0, lim_leve_moderada, facecolor='green', alpha=0.2, label="Somnolencia leve")
    plt.axhspan(lim_leve_moderada, lim_moderada_severa, facecolor='orange', alpha=0.2, label="Somnolencia moderada")
    plt.axhspan(lim_moderada_severa, 1, facecolor='red', alpha=0.2, label="Somnolencia severa")
    plt.plot(perclos, c='k', alpha=0.6)
    colors = map_2_colors(partial_model.predict(data), labels)
    plt.scatter(range(len(perclos)), perclos, c=colors)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

### Random Forest PSD + EOG

In [ ]:
print(f"Experiment: Modelo General Random Forest PSD")
model = RandomForestClassifier(random_state=42, n_jobs=-1)

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

space = dict()
space['n_estimators'] = randint(75, 200)
space['criterion'] = ["gini", "entropy"]
space['max_depth'] = randint(75, 150)
space['min_samples_split'] = randint(2, 10)
space['min_samples_leaf'] = randint(1, 10)
# space['min_weight_fraction_leaf'] = uniform(0, 0.5)
space['max_features'] = ["auto", "sqrt", "log2"]
#space['max_leaf_nodes'] = randint(1, 300)
# space['min_impurity_decrease'] = uniform(0, 5)
#space['bootstrap'] = [True, False]
# space['oob_score'] = [True, False]
space['warm_start'] = [True, False]
# space['class_weight'] = ['balanced', 'balanced_subsample']
#space['ccp_alpha'] = uniform(0, 5)
#space['max_samples'] = uniform(0, 1)

partial_model, acc, pre, rec, f1 = find_and_test_best_model_class(X_sub_eog, y_sub_class, model, space, cv=cv,  n_iter=20)

In [ ]:
attribs = ['psd_delta', 'psd_theta', 'psd_alpha', 'psd_betha', 'psd_gamma', 'n_parpadeos']
importances = partial_model.feature_importances_
importance_features = sorted(zip(importances, attribs), reverse=True)
for imp, attr in importance_features:
    print(f'{attr}: {imp:.4f}')

In [ ]:
for experiment in subset_test:
    key = f'{experiment[:-4]}'
    data = X_all_eog[key]
    
    perclos = y_all[key]
    expected = y_all_class[key]
    
    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    
    print(f'Experiment: {key}')
    
    plt.figure(figsize=(20, 8))
    plt.axhspan(0, lim_leve_moderada, facecolor='green', alpha=0.2, label="Somnolencia leve")
    plt.axhspan(lim_leve_moderada, lim_moderada_severa, facecolor='orange', alpha=0.2, label="Somnolencia moderada")
    plt.axhspan(lim_moderada_severa, 1, facecolor='red', alpha=0.2, label="Somnolencia severa")
    plt.plot(perclos, c='k', alpha=0.6)
    
    predictions = partial_model.predict(data)
    acc = accuracy_score(expected, predictions)
    pre = precision_score(expected, predictions, average='macro', zero_division=1)
    rec = recall_score(expected, predictions, average='macro', zero_division=1)
    f1 = f1_score(expected, predictions, average='macro', zero_division=1)
    print(f"\tAccuracy: {acc*100:.2f}%")
    print(f"\tPrecision: {pre:.4f}")
    print(f"\tRecall: {rec:.4f}")
    print(f"\tf1: {f1:.4f}")
    
    colors = map_2_colors(predictions, labels)
    plt.scatter(range(len(perclos)), perclos, c=colors)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

# Modelo 70-30
- 16/23 = ~70%
- 7/23 = ~30%X_all_eog

In [ ]:
# Separamos 2 experimentos para hacer las pruebas
subset_train, subset_test = train_test_split(features_files, test_size=7, random_state=77)

X_sub = []
X_sub_eog = []
X_sub_eeg = []
y_sub = []
y_sub_class = []

for key in subset_train:
    X_sub.append(X_all[key[:-4]])
    X_sub_eog.append(X_all_eog[key[:-4]])
    aux, aux2 = read_features(f'./features/{key[:-4]}.csv')
    aux_minmax = scaler.fit_transform(aux)
    X_sub_eeg.append(aux_minmax)
    
    y_sub.append(y_all[key[:-4]])
    y_sub_class.append(y_all_class[key[:-4]])

X_sub = np.array(X_sub)
X_sub = X_sub.reshape((X_sub.shape[0]*X_sub.shape[1], X_sub.shape[2]))
X_sub_eog = np.array(X_sub_eog)
X_sub_eog = X_sub_eog.reshape((X_sub_eog.shape[0]*X_sub_eog.shape[1], X_sub_eog.shape[2]))
X_sub_eeg = np.array(X_sub_eeg)
X_sub_eeg = X_sub_eeg.reshape((X_sub_eeg.shape[0]*X_sub_eeg.shape[1], X_sub_eeg.shape[2]))

y_sub = np.array(y_sub)
y_sub = y_sub.reshape((y_sub.shape[0]*y_sub.shape[1]))
y_sub_class = np.array(y_sub_class)
y_sub_class = y_sub_class.reshape((y_sub_class.shape[0]*y_sub_class.shape[1]))

In [ ]:
print(f'{X_sub.shape}, {X_sub_eog.shape}, {X_sub_eeg.shape}, {y_sub.shape}, {y_sub_class.shape}')

## SMV-c

In [ ]:
from sklearn.svm import SVC

### SMV-c PSD

In [ ]:
print(f"Experiment: Modelo 70-30 SVM-r PSD")
model = SVC(random_state=42)

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

space = dict()
space['C'] = uniform(1, 20)
space['kernel'] = ['linear', 'rbf']
space['gamma'] = ['scale', 'auto']
space['shrinking'] = [True, False]
space['probability'] = [True, False]
space['decision_function_shape'] = ['ovo', 'ovr']

partial_model, acc, pre, rec, f1 = find_and_test_best_model_class(X_sub, y_sub_class, model, space, cv=cv,  n_iter=10)

In [ ]:
for experiment in subset_test:
    key = f'{experiment[:-4]}'
    data = X_all[key]
    
    perclos = y_all[key]
    expected = y_all_class[key]
    
    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    
    print(f'Experiment: {key}')
    
    plt.figure(figsize=(20, 8))
    plt.axhspan(0, lim_leve_moderada, facecolor='green', alpha=0.2, label="Somnolencia leve")
    plt.axhspan(lim_leve_moderada, lim_moderada_severa, facecolor='orange', alpha=0.2, label="Somnolencia moderada")
    plt.axhspan(lim_moderada_severa, 1, facecolor='red', alpha=0.2, label="Somnolencia severa")
    plt.plot(perclos, c='k', alpha=0.6)
    
    predictions = partial_model.predict(data)
    acc = accuracy_score(expected, predictions)
    pre = precision_score(expected, predictions, average='macro', zero_division=1)
    rec = recall_score(expected, predictions, average='macro', zero_division=1)
    f1 = f1_score(expected, predictions, average='macro', zero_division=1)
    print(f"\tAccuracy: {acc*100:.2f}%")
    print(f"\tPrecision: {pre:.4f}")
    print(f"\tRecall: {rec:.4f}")
    print(f"\tf1: {f1:.4f}")
    
    colors = map_2_colors(predictions, labels)
    plt.scatter(range(len(perclos)), perclos, c=colors)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

### SMV-r PSD + EOG

In [ ]:
print(f"Experiment: Modelo 70-30 SVM-r PSD+EOG")
model = SVC(random_state=42)

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

space = dict()
space['C'] = uniform(1, 20)
space['kernel'] = ['linear', 'rbf']
space['gamma'] = ['scale', 'auto']
space['shrinking'] = [True, False]
space['probability'] = [True, False]
space['decision_function_shape'] = ['ovo', 'ovr']

partial_model, acc, pre, rec, f1 = find_and_test_best_model_class(X_sub_eog, y_sub_class, model, space, cv=cv,  n_iter=10)

In [ ]:
for experiment in subset_test:
    key = f'{experiment[:-4]}'
    data = X_all_eog[key]
    
    perclos = y_all[key]
    expected = y_all_class[key]
    
    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    
    print(f'Experiment: {key}')
    
    plt.figure(figsize=(20, 8))
    plt.axhspan(0, lim_leve_moderada, facecolor='green', alpha=0.2, label="Somnolencia leve")
    plt.axhspan(lim_leve_moderada, lim_moderada_severa, facecolor='orange', alpha=0.2, label="Somnolencia moderada")
    plt.axhspan(lim_moderada_severa, 1, facecolor='red', alpha=0.2, label="Somnolencia severa")
    plt.plot(perclos, c='k', alpha=0.6)
    
    predictions = partial_model.predict(data)
    acc = accuracy_score(expected, predictions)
    pre = precision_score(expected, predictions, average='macro', zero_division=1)
    rec = recall_score(expected, predictions, average='macro', zero_division=1)
    f1 = f1_score(expected, predictions, average='macro', zero_division=1)
    print(f"\tAccuracy: {acc*100:.2f}%")
    print(f"\tPrecision: {pre:.4f}")
    print(f"\tRecall: {rec:.4f}")
    print(f"\tf1: {f1:.4f}")
    
    colors = map_2_colors(predictions, labels)
    plt.scatter(range(len(perclos)), perclos, c=colors)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

### KNN EEG

In [ ]:
print(f"Experiment: Modelo 70-30 KNN PSD")
model = KNeighborsClassifier()

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

space = dict()
space['n_neighbors'] = randint(3, 21)
space['weights'] = ['uniform', 'distance']
space['algorithm'] = ['auto', 'ball_tree', 'kd_tree', 'brute']
space['leaf_size'] = randint(15,50)
space['p'] = [1, 2]

partial_model, acc, pre, rec, f1 = find_and_test_best_model_class(X_sub_eeg, y_sub_class, model, space, cv=cv,  n_iter=20)

In [ ]:
for experiment in subset_test:
    key = f'{experiment[:-4]}'
    data, aux2 = read_features(f'./features/{key}.csv')
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_minmax = scaler.fit_transform(data)
    
    perclos = y_all[key]
    expected = y_all_class[key]
    
    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    
    print(f'Experiment: {key}')
    
    plt.figure(figsize=(20, 8))
    plt.axhspan(0, lim_leve_moderada, facecolor='green', alpha=0.2, label="Somnolencia leve")
    plt.axhspan(lim_leve_moderada, lim_moderada_severa, facecolor='orange', alpha=0.2, label="Somnolencia moderada")
    plt.axhspan(lim_moderada_severa, 1, facecolor='red', alpha=0.2, label="Somnolencia severa")
    plt.plot(perclos, c='k', alpha=0.6)
    
    predictions = partial_model.predict(data_minmax)
    acc = accuracy_score(expected, predictions)
    pre = precision_score(expected, predictions, average='macro', zero_division=1)
    rec = recall_score(expected, predictions, average='macro', zero_division=1)
    f1 = f1_score(expected, predictions, average='macro', zero_division=1)
    print(f"\tAccuracy: {acc*100:.2f}%")
    print(f"\tPrecision: {pre:.4f}")
    print(f"\tRecall: {rec:.4f}")
    print(f"\tf1: {f1:.4f}")
    
    colors = map_2_colors(predictions, labels)
    plt.scatter(range(len(perclos)), perclos, c=colors)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

### KNN PSD

In [ ]:
print(f"Experiment: Modelo 70-30 KNN PSD")
model = KNeighborsClassifier()

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

space = dict()
space['n_neighbors'] = randint(3, 21)
space['weights'] = ['uniform', 'distance']
space['algorithm'] = ['auto', 'ball_tree', 'kd_tree', 'brute']
space['leaf_size'] = randint(15,50)
space['p'] = [1, 2]

partial_model, acc, pre, rec, f1 = find_and_test_best_model_class(X_sub, y_sub_class, model, space, cv=cv,  n_iter=20)

In [ ]:
for experiment in subset_test:
    key = f'{experiment[:-4]}'
    data = X_all[key]
    
    perclos = y_all[key]
    expected = y_all_class[key]
    
    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    
    print(f'Experiment: {key}')
    
    plt.figure(figsize=(20, 8))
    plt.axhspan(0, lim_leve_moderada, facecolor='green', alpha=0.2, label="Somnolencia leve")
    plt.axhspan(lim_leve_moderada, lim_moderada_severa, facecolor='orange', alpha=0.2, label="Somnolencia moderada")
    plt.axhspan(lim_moderada_severa, 1, facecolor='red', alpha=0.2, label="Somnolencia severa")
    plt.plot(perclos, c='k', alpha=0.6)
    
    predictions = partial_model.predict(data)
    acc = accuracy_score(expected, predictions)
    pre = precision_score(expected, predictions, average='macro', zero_division=1)
    rec = recall_score(expected, predictions, average='macro', zero_division=1)
    f1 = f1_score(expected, predictions, average='macro', zero_division=1)
    print(f"\tAccuracy: {acc*100:.2f}%")
    print(f"\tPrecision: {pre:.4f}")
    print(f"\tRecall: {rec:.4f}")
    print(f"\tf1: {f1:.4f}")
    
    colors = map_2_colors(predictions, labels)
    plt.scatter(range(len(perclos)), perclos, c=colors)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

### KNN PSD + EOG

In [ ]:
print(f"Experiment: Modelo 70-30 KNN PSD+EOG")
model = KNeighborsClassifier()

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

space = dict()
space['n_neighbors'] = randint(3, 21)
space['weights'] = ['uniform', 'distance']
space['algorithm'] = ['auto', 'ball_tree', 'kd_tree', 'brute']
space['leaf_size'] = randint(15,50)
space['p'] = [1, 2]

partial_model, acc, pre, rec, f1 = find_and_test_best_model_class(X_sub_eog, y_sub_class, model, space, cv=cv,  n_iter=20)

In [ ]:
for experiment in subset_test:
    key = f'{experiment[:-4]}'
    data = X_all_eog[key]
    
    perclos = y_all[key]
    expected = y_all_class[key]
    
    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    
    print(f'Experiment: {key}')
    
    plt.figure(figsize=(20, 8))
    plt.axhspan(0, lim_leve_moderada, facecolor='green', alpha=0.2, label="Somnolencia leve")
    plt.axhspan(lim_leve_moderada, lim_moderada_severa, facecolor='orange', alpha=0.2, label="Somnolencia moderada")
    plt.axhspan(lim_moderada_severa, 1, facecolor='red', alpha=0.2, label="Somnolencia severa")
    plt.plot(perclos, c='k', alpha=0.6)
    
    predictions = partial_model.predict(data)
    acc = accuracy_score(expected, predictions)
    pre = precision_score(expected, predictions, average='macro', zero_division=1)
    rec = recall_score(expected, predictions, average='macro', zero_division=1)
    f1 = f1_score(expected, predictions, average='macro', zero_division=1)
    print(f"\tAccuracy: {acc*100:.2f}%")
    print(f"\tPrecision: {pre:.4f}")
    print(f"\tRecall: {rec:.4f}")
    print(f"\tf1: {f1:.4f}")
    
    colors = map_2_colors(predictions, labels)
    plt.scatter(range(len(perclos)), perclos, c=colors)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

### Decision Tree PSD

In [ ]:
print(f"Experiment: Modelo 70-30 Decision Tree PSD")
model = DecisionTreeClassifier(random_state=42)

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

space = dict()
space['criterion'] = ["gini", "entropy"]
space['splitter'] = ["best", "random"]
space['max_depth'] = randint(5, 100)
space['min_samples_split'] = randint(2, 10)
#space['min_samples_leaf'] = randint(1, 10)
space['max_features'] = ["auto", "sqrt", "log2"]
#space['ccp_alpha'] = uniform(0, 5) # empeora el ponerlo, el mejor valor es 0.0

partial_model, acc, pre, rec, f1 = find_and_test_best_model_class(X_sub, y_sub_class, model, space, cv=cv,  n_iter=100)

In [ ]:
attribs = ['psd_delta', 'psd_theta', 'psd_alpha', 'psd_betha', 'psd_gamma', 'n_parpadeos']
importances = partial_model.feature_importances_
importance_features = sorted(zip(importances, attribs), reverse=True)
for imp, attr in importance_features:
    print(f'{attr}: {imp:.4f}')

In [ ]:
for experiment in subset_test:
    key = f'{experiment[:-4]}'
    data = X_all[key]
    
    perclos = y_all[key]
    expected = y_all_class[key]
    
    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    
    print(f'Experiment: {key}')
    
    plt.figure(figsize=(20, 8))
    plt.axhspan(0, lim_leve_moderada, facecolor='green', alpha=0.2, label="Somnolencia leve")
    plt.axhspan(lim_leve_moderada, lim_moderada_severa, facecolor='orange', alpha=0.2, label="Somnolencia moderada")
    plt.axhspan(lim_moderada_severa, 1, facecolor='red', alpha=0.2, label="Somnolencia severa")
    plt.plot(perclos, c='k', alpha=0.6)
    
    predictions = partial_model.predict(data)
    acc = accuracy_score(expected, predictions)
    pre = precision_score(expected, predictions, average='macro', zero_division=1)
    rec = recall_score(expected, predictions, average='macro', zero_division=1)
    f1 = f1_score(expected, predictions, average='macro', zero_division=1)
    print(f"\tAccuracy: {acc*100:.2f}%")
    print(f"\tPrecision: {pre:.4f}")
    print(f"\tRecall: {rec:.4f}")
    print(f"\tf1: {f1:.4f}")
    
    colors = map_2_colors(predictions, labels)
    plt.scatter(range(len(perclos)), perclos, c=colors)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

### Decision Tree PSD + EOG

In [ ]:
print(f"Experiment: Modelo 70-30 Decision Tree PSD+EOG")
model = DecisionTreeClassifier(random_state=42)

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

space = dict()
space['criterion'] = ["gini", "entropy"]
space['splitter'] = ["best", "random"]
space['max_depth'] = randint(5, 100)
space['min_samples_split'] = randint(2, 10)
#space['min_samples_leaf'] = randint(1, 10)
space['max_features'] = ["auto", "sqrt", "log2"]
#space['ccp_alpha'] = uniform(0, 5) # empeora el ponerlo, el mejor valor es 0.0

partial_model, acc, pre, rec, f1 = find_and_test_best_model_class(X_sub_eog, y_sub_class, model, space, cv=cv,  n_iter=100)

In [ ]:
attribs = ['psd_delta', 'psd_theta', 'psd_alpha', 'psd_betha', 'psd_gamma', 'n_parpadeos']
importances = partial_model.feature_importances_
importance_features = sorted(zip(importances, attribs), reverse=True)
for imp, attr in importance_features:
    print(f'{attr}: {imp:.4f}')

In [ ]:
for experiment in subset_test:
    key = f'{experiment[:-4]}'
    data = X_all_eog[key]
    
    perclos = y_all[key]
    expected = y_all_class[key]
    
    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    
    print(f'Experiment: {key}')
    
    plt.figure(figsize=(20, 8))
    plt.axhspan(0, lim_leve_moderada, facecolor='green', alpha=0.2, label="Somnolencia leve")
    plt.axhspan(lim_leve_moderada, lim_moderada_severa, facecolor='orange', alpha=0.2, label="Somnolencia moderada")
    plt.axhspan(lim_moderada_severa, 1, facecolor='red', alpha=0.2, label="Somnolencia severa")
    plt.plot(perclos, c='k', alpha=0.6)
    
    predictions = partial_model.predict(data)
    acc = accuracy_score(expected, predictions)
    pre = precision_score(expected, predictions, average='macro', zero_division=1)
    rec = recall_score(expected, predictions, average='macro', zero_division=1)
    f1 = f1_score(expected, predictions, average='macro', zero_division=1)
    print(f"\tAccuracy: {acc*100:.2f}%")
    print(f"\tPrecision: {pre:.4f}")
    print(f"\tRecall: {rec:.4f}")
    print(f"\tf1: {f1:.4f}")
    
    colors = map_2_colors(predictions, labels)
    plt.scatter(range(len(perclos)), perclos, c=colors)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

### Random Forest PSD

In [ ]:
print(f"Experiment: Modelo 70-30 Random Forest PSD")
model = RandomForestClassifier(random_state=42, n_jobs=-1)

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

space = dict()
space['n_estimators'] = randint(75, 200)
space['criterion'] = ["gini", "entropy"]
space['max_depth'] = randint(75, 150)
space['min_samples_split'] = randint(2, 10)
space['min_samples_leaf'] = randint(1, 10)
# space['min_weight_fraction_leaf'] = uniform(0, 0.5)
space['max_features'] = ["auto", "sqrt", "log2"]
#space['max_leaf_nodes'] = randint(1, 300)
# space['min_impurity_decrease'] = uniform(0, 5)
#space['bootstrap'] = [True, False]
# space['oob_score'] = [True, False]
space['warm_start'] = [True, False]
# space['class_weight'] = ['balanced', 'balanced_subsample']
#space['ccp_alpha'] = uniform(0, 5)
#space['max_samples'] = uniform(0, 1)

partial_model, acc, pre, rec, f1 = find_and_test_best_model_class(X_sub, y_sub_class, model, space, cv=cv,  n_iter=20)

In [ ]:
attribs = ['psd_delta', 'psd_theta', 'psd_alpha', 'psd_betha', 'psd_gamma', 'n_parpadeos']
importances = partial_model.feature_importances_
importance_features = sorted(zip(importances, attribs), reverse=True)
for imp, attr in importance_features:
    print(f'{attr}: {imp:.4f}')

In [ ]:
for experiment in subset_test:
    key = f'{experiment[:-4]}'
    data = X_all[key]
    
    perclos = y_all[key]
    expected = y_all_class[key]
    
    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    
    print(f'Experiment: {key}')
    
    plt.figure(figsize=(20, 8))
    plt.axhspan(0, lim_leve_moderada, facecolor='green', alpha=0.2, label="Somnolencia leve")
    plt.axhspan(lim_leve_moderada, lim_moderada_severa, facecolor='orange', alpha=0.2, label="Somnolencia moderada")
    plt.axhspan(lim_moderada_severa, 1, facecolor='red', alpha=0.2, label="Somnolencia severa")
    plt.plot(perclos, c='k', alpha=0.6)
    
    predictions = partial_model.predict(data)
    acc = accuracy_score(expected, predictions)
    pre = precision_score(expected, predictions, average='macro', zero_division=1)
    rec = recall_score(expected, predictions, average='macro', zero_division=1)
    f1 = f1_score(expected, predictions, average='macro', zero_division=1)
    print(f"\tAccuracy: {acc*100:.2f}%")
    print(f"\tPrecision: {pre:.4f}")
    print(f"\tRecall: {rec:.4f}")
    print(f"\tf1: {f1:.4f}")
    
    colors = map_2_colors(predictions, labels)
    plt.scatter(range(len(perclos)), perclos, c=colors)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

### Random Forest PSD + EOG

In [ ]:
print(f"Experiment: Modelo 70-30 Random Forest PSD+EOG")
model = RandomForestClassifier(random_state=42, n_jobs=-1)

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

space = dict()
space['n_estimators'] = randint(75, 200)
space['criterion'] = ["gini", "entropy"]
space['max_depth'] = randint(75, 150)
space['min_samples_split'] = randint(2, 10)
space['min_samples_leaf'] = randint(1, 10)
# space['min_weight_fraction_leaf'] = uniform(0, 0.5)
space['max_features'] = ["auto", "sqrt", "log2"]
#space['max_leaf_nodes'] = randint(1, 300)
# space['min_impurity_decrease'] = uniform(0, 5)
#space['bootstrap'] = [True, False]
# space['oob_score'] = [True, False]
space['warm_start'] = [True, False]
# space['class_weight'] = ['balanced', 'balanced_subsample']
#space['ccp_alpha'] = uniform(0, 5)
#space['max_samples'] = uniform(0, 1)

partial_model, acc, pre, rec, f1 = find_and_test_best_model_class(X_sub_eog, y_sub_class, model, space, cv=cv,  n_iter=20)

In [ ]:
attribs = ['psd_delta', 'psd_theta', 'psd_alpha', 'psd_betha', 'psd_gamma', 'n_parpadeos']
importances = partial_model.feature_importances_
importance_features = sorted(zip(importances, attribs), reverse=True)
for imp, attr in importance_features:
    print(f'{attr}: {imp:.4f}')

In [ ]:
for experiment in subset_test:
    key = f'{experiment[:-4]}'
    data = X_all_eog[key]
    
    perclos = y_all[key]
    expected = y_all_class[key]
    
    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    
    print(f'Experiment: {key}')
    
    plt.figure(figsize=(20, 8))
    plt.axhspan(0, lim_leve_moderada, facecolor='green', alpha=0.2, label="Somnolencia leve")
    plt.axhspan(lim_leve_moderada, lim_moderada_severa, facecolor='orange', alpha=0.2, label="Somnolencia moderada")
    plt.axhspan(lim_moderada_severa, 1, facecolor='red', alpha=0.2, label="Somnolencia severa")
    plt.plot(perclos, c='k', alpha=0.6)
    
    predictions = partial_model.predict(data)
    acc = accuracy_score(expected, predictions)
    pre = precision_score(expected, predictions, average='macro', zero_division=1)
    rec = recall_score(expected, predictions, average='macro', zero_division=1)
    f1 = f1_score(expected, predictions, average='macro', zero_division=1)
    print(f"\tAccuracy: {acc*100:.2f}%")
    print(f"\tPrecision: {pre:.4f}")
    print(f"\tRecall: {rec:.4f}")
    print(f"\tf1: {f1:.4f}")
    
    colors = map_2_colors(predictions, labels)
    plt.scatter(range(len(perclos)), perclos, c=colors)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

# Ahora que tenemos una input mucho mas grande al juntar todos los X en uno, vamos a probar con Deep Learning (GRU)

In [ ]:
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.optimizers import SGD, Adam

In [ ]:
def gru_train_model(X, y, neurons=15, layers=2, verbose=1):
    
    # Tenemos que pasar de X de (885,n_features) a (885,n_features,1) para trabajar con RNN (LSTM, GRU, ...)
    X_rs = np.reshape(X, (X.shape[0],X.shape[1],1))
    # Tenemos que pasar de y de (885,) a (885,1)
    y_rs = np.reshape(y, (y.shape[0],1))

    X_train, X_test, y_train, y_test = train_test_split(X_rs, y_rs, test_size=0.25, random_state=42)

    model = Sequential()
    model.add(GRU(neurons, input_shape=(None, 1), return_sequences=True))
    
    #Internal layers
    for i in range(layers-1):
        model.add(GRU(neurons, return_sequences=True))
    model.add(GRU(neurons))
    
    model.add(Dense(1))

    early_stop = EarlyStopping(monitor='loss', patience=10, verbose=verbose)
    optimizer = Adam(beta_1=0.9, beta_2=0.999, lr=0.01, decay=1e-4)

    model.compile(loss='mean_squared_error', optimizer=optimizer)

    #model.summary()
    
    model.fit(X_train, y_train, batch_size=32, epochs=200, verbose=verbose, callbacks=[early_stop])
    
    return model

In [ ]:
DL_model = gru_train_model(X, y)

In [ ]:
for key in X_all:
    data = X_all[key]
    expected = y_all[key]

    predictions = DL_model.predict(np.reshape(data, (data.shape[0],data.shape[1],1)))
    local_mse = mean_squared_error(np.reshape(expected, (expected.shape[0],1)), prediction)
    
    plt.figure(figsize=(20, 8))
    plt.plot(expected, color="black", label="real")
    plt.plot(predictions, color="red", label="predicted")
    plt.title(f'Exp:{key}, local_MSE:{local_mse:.4f}')
    plt.xlabel('Epoch')
    plt.ylabel('% PERCLOS')
    plt.legend(loc='best')
    plt.ylim(0, 1.1)
    plt.show()